# Table 2&3 (fixed effects regression)

In [28]:
import pandas as pd
import numpy as np
import matplotlib as plt


df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

(16127, 31)


* ***Construct Work Category Dummies***

In [24]:
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#print(df.shape) to check column number (16127,36)
df.head()

,authority_code,year,auth_anno,fpsb_auction,n_bidders,discount,reserve_price,work_category,complex_work,delay_ratio,...,ctrl_exp_turin_co_sample,ctrl_pop_exp_turin_co_sample,ctrl_pop_turin_pr_sample,ctrl_exp_turin_pr_sample,ctrl_pop_exp_turin_pr_sample,OG03_dummy,OG01_dummy,OG_rest_dummy,OG_dummy,OS_dummy
0,543.0,2004.0,1642.0,0.0,3.0,8.150,2.879197e+05,OS24,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
1,543.0,2004.0,1642.0,0.0,3.0,8.150,2.879197e+05,OS24,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
2,543.0,2004.0,1642.0,0.0,3.0,8.150,2.879197e+05,OS24,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
3,3090272.0,2000.0,798.0,0.0,77.0,17.313,8.530979e+05,OG03,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1,0,0,1,0
4,3090272.0,2000.0,798.0,0.0,45.0,16.750,1.257377e+06,OG03,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1,0,0,1,0


* ***Treated vs Controls***

In [25]:
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

* ***PA Specific***

In [27]:
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#본격적으로 df에 id_auth 칼럼을 만들것임
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

#for statement check            
#df.head
#print(max(df['id_path']))

KeyboardInterrupt: 

* regression tools

In [6]:
#from linearmodels import PanelOLS
!pip install econtools
import econtools as econ
import econtools.metrics as mt

In [ ]:
#2001~2010 dummy
for j in range(2001,2011):
    df['_Iyear_'+str(j)] = 0
    for i in range(len(df)):
        if df.loc[i,'year'] == j:
            df.loc[i, '_Iyear_'+str(j)] = 1
            

In [16]:
df.head()

,authority_code,year,auth_anno,fpsb_auction,n_bidders,discount,reserve_price,work_category,complex_work,delay_ratio,...,_Iyear_2004,_Iyear_2001,_Iyear_2002,_Iyear_2003,_Iyear_2005,_Iyear_2006,_Iyear_2007,_Iyear_2008,_Iyear_2009,_Iyear_2010
15628,2.0,2006.0,17.0,0.0,46.0,13.730000,154926.0000,OG03,1.0,NaN,...,0,0,0,0,0,1,0,0,0,0
1830,2.0,2003.0,15.0,0.0,30.0,17.209999,252500.0000,OS28,2.0,10.0,...,0,0,0,1,0,0,0,0,0,0
14964,2.0,2008.0,19.0,0.0,68.0,21.049999,281850.9375,OG03,1.0,NaN,...,0,0,0,0,0,0,0,1,0,0
1731,2.0,2003.0,15.0,0.0,25.0,14.400000,387800.0000,OG01,2.0,0.0,...,0,0,0,1,0,0,0,0,0,0
15544,2.0,2008.0,19.0,0.0,23.0,22.799999,623500.0000,OG03,1.0,NaN,...,0,0,0,0,0,0,0,1,0,0


In [9]:
#treatment ; "turin_co_sample turin_pr_sample" == 1
#ctrl_exp_`t' == 1 

#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262

#df_reg_dis = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()
#mt.reg(df_reg_dis, 'fpsb_auction','discount', fe_name = 'authority_code', cluster = 'auth_anno')
#fe_name = <fe_var>; fixed effect

SyntaxError: unexpected EOF while parsing (<ipython-input-9-7d36911b743c>, line 16)

In [17]:
import statsmodels.api as sm

'''X = df[[“RM”, “LSTAT”]]
y = target[“MEDV”]
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()'''

'X = df[[“RM”, “LSTAT”]]\ny = target[“MEDV”]\nmodel = sm.OLS(y, X).fit()\npredictions = model.predict(X)\nmodel.summary()'

In [22]:
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262
df_reg_dis = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()
#X1 = df[['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']]
#y = df['discount']
#model = sm.OLS(y,X1).fit()
#predictions = model.predict(X1)
#model.summary()


SyntaxError: unexpected EOF while parsing (<ipython-input-22-40f01a120c70>, line 7)